In [1]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving devices.csv to devices.csv
Saving energy_logs.csv to energy_logs.csv
Saving rooms.csv to rooms.csv


In [16]:
energy_logs = pd.read_csv(list(uploaded.keys())[1])
print(energy_logs)
devices = pd.read_csv(list(uploaded.keys())[0])
print(devices)
rooms = pd.read_csv(list(uploaded.keys())[2])
print(rooms)

   log_id  device_id            timestamp  energy_kwh
0       1          1  2025-07-22 06:30:00        0.55
1       2          2  2025-07-22 07:15:00        1.80
2       3          3  2025-07-22 08:45:00        2.25
   device_id          name  room_id    status
0          1    Smart Lamp        1    active
1          2  Air Purifier        2  inactive
2          3  Water Heater        3    active
   room_id        name
0        1  Study Room
1        2  Guest Room
2        3    Bathroom


In [21]:
print("\n Cleaning Data...")
energy_logs['timestamp'] = pd.to_datetime(
    energy_logs['timestamp'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

energy_logs['energy_kwh'] = pd.to_numeric(energy_logs['energy_kwh'], errors='coerce')

energy_logs.dropna(subset=['timestamp', 'energy_kwh', 'device_id'], inplace=True)

print("\n Cleaned Energy Logs:")
print(energy_logs.head())


 Cleaning Data...

 Cleaned Energy Logs:
   log_id  device_id           timestamp  energy_kwh
0       1          1 2025-07-22 06:30:00        0.55
1       2          2 2025-07-22 07:15:00        1.80
2       3          3 2025-07-22 08:45:00        2.25


In [22]:
print("\n Generating Device-level Summary...")
device_summary = energy_logs.groupby('device_id')['energy_kwh'].agg(['sum','mean']).reset_index()
device_summary.rename(columns={'sum':'total_energy_kwh','mean':'avg_energy_kwh'}, inplace=True)

print("\n Device-level Energy Summary:")
print(device_summary)


 Generating Device-level Summary...

 Device-level Energy Summary:
   device_id  total_energy_kwh  avg_energy_kwh
0          1              0.55            0.55
1          2              1.80            1.80
2          3              2.25            2.25


In [23]:
print("\n Merging Energy Logs with Device -> Room Mapping...")
energy_logs = energy_logs.merge(devices[['device_id','room_id']], on='device_id', how='left')
print("\n Merged Energy Logs:")
print(energy_logs.head())


 Merging Energy Logs with Device -> Room Mapping...

 Merged Energy Logs:
   log_id  device_id           timestamp  energy_kwh  room_id
0       1          1 2025-07-22 06:30:00        0.55        1
1       2          2 2025-07-22 07:15:00        1.80        2
2       3          3 2025-07-22 08:45:00        2.25        3


In [24]:
print("\n Generating Room-level Summary...")
room_summary = energy_logs.groupby('room_id')['energy_kwh'].sum().reset_index()
room_summary = room_summary.merge(rooms, on='room_id', how='left')

print("\n Room-level Energy Summary:")
print(room_summary)


 Generating Room-level Summary...

 Room-level Energy Summary:
   room_id  energy_kwh        name
0        1        0.55  Study Room
1        2        1.80  Guest Room
2        3        2.25    Bathroom


In [26]:

energy_logs.to_csv("cleaned_energy_usage.csv", index=False)
print("\n💾 Cleaned dataset saved as 'cleaned_energy_usage.csv'")



💾 Cleaned dataset saved as 'cleaned_energy_usage.csv'


In [27]:
from google.colab import files
files.download("cleaned_energy_usage.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>